In [2]:
# default_exp core

In [1]:
!sh setup.sh

  Using cached fastai-2.5.2-py3-none-any.whl (186 kB)
  Using cached spacy-3.1.3-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached spacytextblob-3.0.1-py3-none-any.whl (4.1 kB)
     |████████████████████████████    | 728.6 MB 79.5 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 831.4 MB 11 kB/s s eta 0:00:01
     |████████████████████████████████| 11.5 MB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 54.8 MB/s eta 0:00:01
     |████████████████████████████████| 22.1 MB 47.6 MB/s eta 0:00:01
     |████████████████████████████████| 22.2 MB 59.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 52.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 64.1 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 53.7 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 456 kB 69.1 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 76 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 625 kB 48.5 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 49.2 MB/s eta 0:00:01
     |███████████████████████

In [7]:
# def find_sentence_in_abstract(paragraph, cue_phrases):
#     """checks paragraph for key sentences. Returns first matching hit"""
#     p_sents = paragraph.split(".")
#     combinations = itertools.product(p_sents, cue_phrases)
#     hit = ""
#     for c in combinations:
#         if c[1] in c[0]:
#        # if c[0].startswith(c[1]): #easier to work with later assuming beginning of sentence
#             hit = c[0]
#             break
#     return hit

In [8]:
#export

import json
import re


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [9]:
#!pip install biopython pydash

In [10]:
#export
import pandas as pd
from Bio import Entrez
from pydash import get

def get_pubmed_records(pmids):
    Entrez.email = "strasser.ms@gmail.com"
    # handle type is http.client.HTTPResponse
    handle = Entrez.efetch(
        db="pubmed",
        id=",".join(pmids),
        #api_key="be67e0a1be023d17fff5334a6c6f45287a08",
        rettype="xml",
        retmode="text",
    )
    records = Entrez.read(handle)
    return records

def get_attribute_text(AbstractText, NlmCategory):
    if AbstractText is None:
        return None
    for string_element in AbstractText:
        cat = ""
        try:
            #the worst API arbitrary nonsense ... 1h of my life for getting data out of it
            cat = string_element.attributes["NlmCategory"]
        except:
            None
        
        if cat == NlmCategory:
            return string_element

def get_segment(record, segment_label):
    if record is None:
        return None
    return str(
        get_attribute_text(
            get(record, "MedlineCitation.Article.Abstract.AbstractText"), 
            segment_label
        )
    )

In [11]:
#export
import itertools

#OLD one-off functions that need refactoring
def sentence_has_phrase(sentence, cue_phrases):
    for c in cue_phrases: #unsafe scope
        if c in sentence:
            return True
    return False
    
def find_sentence_in_abstract(paragraph, bias=0):
    """checks paragraph for key sentences. Returns first matching hit"""
    p_sents = split_into_sentences(paragraph)
    idx = find_index(p_sents, sentence_has_phrase)
    if idx < 0:
        return ""
    
    i = idx + bias #bias- for sentence before or after
    if i < 0:
        return ""
    return get(p_sents, i)

In [12]:
import requests as req

# url = "https://github.com/derekchuank/high-frequency-vocabulary/blob/master/30k.txt"
# res = req.get(url)
# vocab30k = res.text.split("\t\n")
# vocab30k[:5]

In [13]:
#export

import spacy
import en_core_web_md
with open("vocab30k.txt") as f:
    vocab30k = f.read().split("\t\n")

nlp = en_core_web_md.load()

extra_vocab = [
               "-PRON-", "-pron-", ".", ",", ";"  #lemmatizer inconsitency
]

special_tokens = {
    "(": "-lrb-",
    ")": "-rrb-"
}

def replace_outof_vocab_words(text, vocab, nlp=nlp, extra_vocab=extra_vocab, special_tokens=special_tokens):
    vocab += extra_vocab
    newtext = ""
    for token in nlp(text):
        t = token.text
        if special_tokens.get(t):
            t=special_tokens.get(t)
        elif token.lemma_.lower() not in vocab and token.tag_ is not None:
            t= "ii" + token.tag_.lower()
        newtext += t + " "

    return newtext
    


In [14]:
#replace_outof_vocab_words('Pancreaticobiliary maljunction (PBM) refers to the union of the pancreatic and biliary ducts outside of the duodenal wall.')

In [15]:
#NLTK
patterns = [
    (r'.*ing$', 'VBG'),               # gerunds
    (r'.*ed$', 'VBD'),                # simple past
    (r'.*es$', 'VBZ'),                # 3rd singular present
    (r'.*ould$', 'MD'),               # modals
    (r'.*\'s$', 'NN$'),               # possessive nouns
    (r'.*s$', 'NNS'),                 # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'.*', 'NN')                     # nouns (default)
]

## Pubmed Abstract/Fulltext Fetching and Cleaning

Pubmed does not give fulltext/abstract text in a good format. It gives a CSV with the metadata of searched results. That means you have to use the Entrez fetch and then merge the abstracts and text information with the metadata to have something workable

In [ ]:
with open("clinical_study_abst.txt") as f:
    text = f.read()
    
df = pd.read_csv("clin_study_meta.csv")
records = get_pubmed_records([str(pmid) for pmid in df["PMID"]])
len([r for r in records["PubmedArticle"] if r is not None])

articles = records["PubmedArticle"]

g = {
    "PMID": [str(get(r,"MedlineCitation.PMID")) for r in articles],
    
    "BACKGROUND": [get_segment(r, "BACKGROUND") for r in articles],
    "OBJECTIVE": [get_segment(r, "OBJECTIVE") for r in articles],
    "METHODS": [get_segment(r, "METHODS") for r in articles],
    "RESULTS": [get_segment(r, "RESULTS") for r in articles],
    "CONCLUSIONS": [get_segment(r, "CONCLUSIONS") for r in articles]
     }

df2 = pd.DataFrame(g)
df2["PMID"] = df2["PMID"].astype(int)
dfx = pd.merge(df, df2, on="PMID")
dfx.to_csv("enriched_pm_10k.csv")

dfx.head(), len(df), len(df2), df[:6768]["PMID"][0], df2[:6768]["PMID"][0]

## Text Features with spacy extensions

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.polarity      # Polarity: -0.125
doc._.subjectivity  # Sujectivity: 0.9
doc._.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]